In [4]:
import os
from onekey_algo import get_param_in_cwd
from glob import glob
import pandas as pd

POSSIBLE_IMG = ['.jpg', '.jpeg', '.png', '.bmp']

data_root = get_param_in_cwd('radio_dir')
train = pd.read_csv(os.path.join('split_info', 'train-RND-0.txt'), sep='\t', header=None, names=['fname', 'label'])
test = pd.read_csv(os.path.join('split_info', 'val-RND-0.txt'), sep='\t', header=None, names=['fname', 'label'])
already = set([os.path.basename(s) for s in set(train['fname']) | set(test['fname'])])
all_samples = glob(os.path.join(data_root, 'crop/*.png'))
samples = [s for s in all_samples if (os.path.basename(s) not in already)]
print(len(all_samples), len(samples))

3474 97


In [11]:
import json
from onekey_algo.classification.eval_classification import init as clf_init
from onekey_algo.classification.eval_classification import inference_dataloader as clf_inference

model_name = 'resnet18'
print(f"正在预测{model_name}")
config_path = os.path.join(data_root, 'models', 'CV-0', model_name, 'viz')
model, transformer, labels, device = clf_init(config_path=config_path)
results = clf_inference(samples, model, transformer, labels, device, num_workers=4)

prediction = []
for result in results:
    pred = json.loads(result[1])
    prediction.append([result[0], pred['1'], result[2], 1])

prediction = pd.DataFrame(prediction, columns=['fpath', 'prob', 'pred', 'gt'])
prediction.to_csv(os.path.join(data_root, 'models', 'CV-0', model_name, 'viz', 'aux_samples.txt'), sep='\t', index=False, header=False)
display(prediction)
# print(json.dumps(results, ensure_ascii=False, indent=True))

正在预测resnet18


,fpath,prob,pred,gt
0,D:\\20240908-Vince\crop\primary-236.nii.png,0.559,1,1
1,D:\\20240908-Vince\crop\primary-236.nii_+01.png,0.527,1,1
2,D:\\20240908-Vince\crop\primary-236.nii_+02.png,0.409,0,1
3,D:\\20240908-Vince\crop\primary-236.nii_+04.png,0.569,1,1
4,D:\\20240908-Vince\crop\primary-236.nii_-01.png,0.514,1,1
...,...,...,...,...
92,D:\\20240908-Vince\crop\secondary2-50.nii_+02.png,0.794,1,1
93,D:\\20240908-Vince\crop\secondary2-50.nii_+04.png,0.790,1,1
94,D:\\20240908-Vince\crop\secondary2-50.nii_-01.png,0.477,0,1
95,D:\\20240908-Vince\crop\secondary2-50.nii_-02.png,0.361,0,1
